In [1]:
import numpy as np
import pandas as pd

In [2]:
def read_datafile(filename):
    df = pd.read_csv(filename,delimiter='\s+')
    X = df.values
    return X

In [3]:
X = read_datafile('blobs.txt')

In [5]:
def kth_nearest_neighbour(datapoint,X,k):
    arr = []
    for x in X:
        arr.append(np.linalg.norm(datapoint-X))
        
    return sorted(arr)[k-1]

In [ ]:
def generate_all_kth_nearest_neightbour(X,k):
    arr = []
    
    for each in X:
        arr.append(kth_nearest_neighbour())